### Figure 3C - Early vs Late Capsule Scatter Plot

Here we look at a read by read scatter plot of the early capsule sample vs the late capsule sample to see if the encapsulated reads have changed significantly while the background cellular context is changing. 

In [4]:
import pandas as pd
import os
from bokeh.io import push_notebook, show, output_notebook, export_svg
output_notebook()
from bokeh.plotting import figure

Loading BokehJS ...

First we will read our data frames into a file:

In [5]:
df_dir = '../../../Processed Data Files/dataframe files/230322Li/'
df_dict = dict()
names = []
# Fetch all the relevant dataframe files and put them in a useable format:
for file in os.listdir(df_dir):
    if file.endswith('_dataframe.txt'):
        name = file[:]
        name = name[:name.find('_2')]
        new_df  = pd.read_csv(''.join([df_dir,file]))
        new_df['Length'] = new_df['Stop'] - new_df['Start']
        df_dict[name] = new_df
        names.append(name)
print(f'\nHere is the list of unique samples for which there are dataframe files: \n\n{names}\n')

# a name dictionary for converting between the datafram name and a form nicer for plots:
names_dict = {'Before-Ind': 'Before Induction', 'Early-Cap': "Early Capsule",
              'Early-Lys': "Early Lysate", 'Late-Cap': "Late Capsule",
              'Late-Lys': "Late Lysate", 'Mid-Stress': "Mid-Stress, Total Lysate",
              'No-Ind': "No Induction", 'Post-Ind': "Post Induction, Total Lysate"}


Here is the list of unique samples for which there are dataframe files: 

['Before-Ind', 'Early-Cap', 'Early-Lys', 'Late-Cap', 'Late-Lys', 'Mid-Stress', 'No-Ind', 'Post-Ind']



Some helper functions to help us with the plotting:

In [6]:
def into_RPM(df_dict, name):
    '''This felper function gets the dataframe for a particular experiment and converts its reads into RPM'''
    sample_df = df_dict[name].copy()
    sample_df = sample_df.rename(columns = {'Counts':name})
    sample_df[name] = sample_df[name]/(sum(sample_df[name])/1000000) #RPM]
    
    return sample_df

Now we will plot a simple scatter plot comparing the early and late Lysate samples on a read by read basis:

In [24]:
x = 'Early-Cap' # Early encapsulated reads
y = 'Late-Cap' # Late encapsulated reads

x_df = into_RPM(df_dict, x)
y_df = into_RPM(df_dict, y)
plot_df = x_df.merge(y_df)
plot_df.drop(plot_df[plot_df['Name'] == 'error'].index, inplace = True) # currently dropping any gene without reads in both
plot_df.drop(plot_df[plot_df[x] == 0].index, inplace = True)
plot_df.drop(plot_df[plot_df[y] == 0].index, inplace = True)

p = figure(
    y_axis_type = "log", x_axis_type = "log",
    aspect_scale = 1, width = 1000, height = 1000,
    output_backend = "svg", tooltips = [("Gene", "@Name")]
)

#configure title appearance:
p.title = "RPM of Early and Late Capsule Samples"
p.title.text_font = "arial"
p.title.text_font_size = "30px"
p.title.align = "center"

p.xaxis.axis_label = "Early Capsule RPM"
p.yaxis.axis_label = "Late Capsule RPM"
p.axis.ticker = [10**0, 10**2, 10**4, 10**6]
p.axis.major_label_text_font = "arial"
p.axis.major_label_text_font_style = "bold"
p.axis.major_label_text_font_size = "25px"
p.axis.axis_label_text_font = "arial"
p.axis.axis_label_text_font_style = "bold"
p.axis.axis_label_text_font_size = "30px"

p.background_fill_color = "whitesmoke"
p.grid.grid_line_color = "white"
p.grid.grid_line_width = 4

p.circle(x=x, y=y, source=plot_df, size=15, fill_alpha=0.8, line_alpha=0, color="darkslateblue",)
p.line(x=[10**(-1), 10**6], y=[10**(-1), 10**6], color = "darkseagreen", width = 4, legend_label = "x = y")
p.line(x=[10*10**(-1), 10*10**6], y=[10**(-1), 10**6], color = "orangered", width = 4, legend_label = "x = 10y")
p.line(x=[.1*10**(-1), .1*10**6], y=[10**(-1), 10**6], color = "mediumvioletred", width = 4, legend_label = "x = .1y")

p.legend.location = (100, 550)
p.legend.label_text_font_size = '35pt'
show(p)
export_svg(p, filename = './3C.svg')

['./3C.svg']